In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import scipy
import cdb_extras.xarray_support as cdbxr
from pyCDB import client
from scipy.signal import find_peaks
cdb = client.CDBClient()

In [2]:
def calculate_gradient(x1, y1, x2, y2):

    gradient = (y2 - y1) / (x2 - x1)
    return gradient

In [3]:
def Load_Signal(shot_number):
    
    shot_number = shot_number
    W_xarray = 1
    t_ELM_start= 1
    signal = 1
    
    shot_accessor = cdbxr.Shot(shot_number)

    Variants = ["HIRES_ELM", "v7_std_hires"]
    DefaultVariant = "default_variant"

    try:
        for variant in Variants:
            signal_name = f'W/EFIT:{shot_number}:{variant}'
            try:
                W_xarray = shot_accessor[signal_name]
                print(" ")
               
                break 
            except Exception:
                print(" ")
        else:
            
            try:
                default_signal_name = f'W/EFIT:{shot_number}'
                W_xarray = shot_accessor[default_signal_name]
                print(" ")
                
            except Exception:
                print(" ")
                
    except Exception as e:
        print(" ", e)
    
    try:
        t_ELM_start = cdb.get_signal("t_ELM_start:"+ str(shot_number))
    except Exception:
        print(" ")

    try:
        signal = cdb.get_signal(f"t_ELM_start/SYNTHETIC_DIAGNOSTICS:{shot_number}")
    except Exception:
        print(" ")
        
        
    Signal_Data=[W_xarray,t_ELM_start,signal]
        
    return Signal_Data 

In [4]:
def Get_Lists(shot_number):
    
    Signal_Data = Load_Signal(shot_number)
    
    W_xarray = Signal_Data[0]
    t_ELM_start = Signal_Data[1]
    signal = Signal_Data[2]
    
    width = 15
    distance = 5
    rel_height = 1
    
    UPeaks = []
    UTroughs = []
    UPeaks_T = []
    UTroughs_T =[]
               
    FPeaks = []
    FTroughs = []
    FPeaks_T = []
    FTroughs_T =[]
    
    peaks, _ = find_peaks(W_xarray, width = width, distance = distance, rel_height=rel_height) 
    peaks2, _ = find_peaks(-W_xarray, width = width, distance = distance, rel_height=rel_height)
    
    Peaks_T = list(W_xarray.time.data[peaks])
    Peaks = list(W_xarray.data[peaks])
    Troughs_T = list(W_xarray.time.data[peaks2])
    Troughs = list(W_xarray.data[peaks2])
    
    elm_candidates = list()
    for i in range(len(Peaks_T)):
        threshold_time = Peaks_T[i]
        for j, through_time in enumerate(Troughs_T):
            if through_time > threshold_time and (i == len(Peaks_T)-1 or Peaks_T[i+1] > through_time):
                elm_candidates.append((threshold_time, through_time))
                break
    
    t_starts = [elm_c[0] for elm_c in elm_candidates]
    t_end = [elm_c[1] for elm_c in elm_candidates]

    for t_s, t_e in zip(t_starts, t_end):
        UPeaks_T.append( Peaks_T [ Peaks_T.index( t_s ) ] )
        UTroughs_T.append(Troughs_T[Troughs_T.index(t_e)])
        UPeaks.append(Peaks[Peaks_T.index(t_s)])
        UTroughs.append(Troughs[Troughs_T.index(t_e)])
    
    for i in range(len(UPeaks_T)):
        if calculate_gradient(UPeaks_T[i], UPeaks[i], UTroughs_T[i], UTroughs[i]) < -100 and UTroughs[i]/UPeaks[i]*100<99:
        
            FPeaks_T.append(UPeaks_T[i])
            FTroughs_T.append(UTroughs_T[i])
            FPeaks.append(UPeaks[i])
            FTroughs.append(UTroughs[i])
    
    Temp_List = []
    Temp_List.append(FPeaks)
    Temp_List.append(FPeaks_T)
    Temp_List.append(FTroughs)
    Temp_List.append(FTroughs_T)
    
    return Temp_List

In [5]:
def Display_Graph(shot_number):
    
    Signal_Data = Load_Signal(shot_number)
    
    W_xarray = Signal_Data[0]
    t_ELM_start = Signal_Data[1]
    signal = Signal_Data[2]
    
    Lists = Get_Lists(shot_number)
    
    NPeaks = Lists[0]
    NPeaks_T = Lists[1]
    NTroughs = Lists[2]
    NTroughs_T = Lists[3]
    
    plt.figure()
    W_xarray.plot(color = "black")
    plt.plot(NPeaks_T,NPeaks,'xr')

In [6]:
def Compare_Graphs(shot_number):
    
    Signal_Data = Load_Signal(shot_number)
    
    W_xarray = Signal_Data[0]
    t_ELM_start = Signal_Data[1]
    signal = Signal_Data[2]
    
    Lists = Get_Lists(shot_number)
    
    ELM_start_J = Lists[1]
    ELM_start_T = signal.data
    
    ELM_start_J1 = []
    ELM_start_T1 = []

    ELM_start_J2 = []
    ELM_start_T2 = []
    
    elm_candidates = list()

    for threshold_time in ELM_start_J:
        closest_through_time = None
        min_time_difference = float('inf')
        for through_time in ELM_start_T:
            time_difference = abs(through_time - threshold_time)
            if time_difference < min_time_difference:
                min_time_difference = time_difference
                closest_through_time = through_time
        if closest_through_time is not None:
            elm_candidates.append((threshold_time, closest_through_time))

    t_starts = [elm_c[0] for elm_c in elm_candidates]
    t_end = [elm_c[1] for elm_c in elm_candidates]
    
    for t_s, t_e in zip(t_starts, t_end):
        ELM_start_J1.append(t_s)
        ELM_start_T1.append(t_e)
    
    for i in range(len(ELM_start_J1)):
        if ELM_start_J1[i]-ELM_start_T1[i] < 1 and ELM_start_J1[i]-ELM_start_T1[i] > -1:
            ELM_start_J2.append(ELM_start_J1[i])
            ELM_start_T2.append(ELM_start_T1[i])
    
    print(f"No. of ELMs using Plasma Energy: {len(ELM_start_J)}. No. of ELMs using H_Alpha: {len(ELM_start_T)}. No. of ELMs from anaylsis: {len(ELM_start_J2)}.")
    print("| Rough Time of ELM from Plasma Energy (ms) | Rough Time of ELM from H_Alpha (ms) | Difference (ms) |") 
    print("-----------------------------------------------------------------------------------------------------")
    
    for i in range(len(ELM_start_J2)): 
        First_Print1 = "| {:^41.2f} | {:^35.2f} | {:^15.2f} |".format(ELM_start_J2[i], ELM_start_T2[i], (ELM_start_J2[i] - ELM_start_T2[i])) 
        print(First_Print1)

In [7]:
# fig, (ax1, ax2) = plt.subplots(2, sharex=True)
# fig.suptitle('Aligning x-axis using sharex')
# ax1.plot(signal.time_axis.data, -signal.data)
# ax2.plot(W_xarray.time.data, W_xarray.data)
# ax1.vlines(t_ELM_start.data, 0, 6, color='red')
# ax2.vlines(t_ELM_start.data, 0, 7000, color='red')
# plt.plot(FPeaks_T,FPeaks,'xg')

# ax1.grid()
# ax2.grid()

In [8]:
running = True

shot_number = int(input("Enter Shot Number: ")) 

print("| Start Time (ms) | End Time (ms) | Duration (ms) | Energy max (J) | Energy min (J) | Energy difference (J) |") 
print("-------------------------------------------------------------------------------------------------------------")

List = Get_Lists(shot_number)
    
GPeaks = List[0]
GPeaks_T = List[1]
GTroughs = List[2]
GTroughs_T = List[3]

for i in range(len(GPeaks_T)): 
    First_Print = "| {:^15.2f} | {:^13.2f} | {:^13.2f} | {:^14.2f} | {:^14.2f} | {:^21.2f} |".format(GPeaks_T[i], GTroughs_T[i], (GTroughs_T[i] - GPeaks_T[i]), GPeaks[i], GTroughs[i], (GPeaks[i]-GTroughs[i])) 
    print(First_Print)

while running: 

    print("Would you like to compare against another ELM Detection Algorithim, look at the ELMs on a graph or exit?")
    Var = input("Type ESC - to exit, GRAPH - to see the graph or COMPARE - to comapre ELMs found in here, and ELM's found using another signal: ")
    
    Var = Var.upper()

    if Var == "GRAPH": 
        Display_Graph(shot_number)
      
    if Var == "COMPARE":
        Compare_Graphs(shot_number)

    if Var == "ESC":
        running = False
        break

ValueError: invalid literal for int() with base 10: ''